## Metis Project 2 - Predicting Market Values of Soccer Players in the Top 5 European Leagues Using Linear Regression

### Creating a Dataframe of Player Market Values (Target Variable)

In [1]:
import scrape_utils as su
import pandas as pd
import numpy as np

#### Premier League player market values

In [2]:
pmv_table_epl = su.concat_table_epl().reset_index(drop=True)
pmv_table_epl

,Player Name,Current Market Value (USD)
0,Harry Kane,132000000.0
1,Mohamed Salah,121000000.0
2,Trent Alexander-Arnold,110000000.0
3,Sadio Mané,110000000.0
4,Raheem Sterling,110000000.0
...,...,...
559,Richard Nartey,220000.0
560,Elia Caprile,110000.0
561,Rhys Williams,5500000.0
562,Owen Otasowie,1650000.0


#### La Liga player market values

In [3]:
pmv_table_laliga = su.concat_table_laliga().reset_index(drop=True)
pmv_table_laliga

,Player Name,Current Market Value (USD)
0,Frenkie de Jong,88000000.0
1,João Félix,88000000.0
2,Pedri,77000000.0
3,Marcos Llorente,77000000.0
4,José María Giménez,66000000.0
...,...,...
526,Pablo Piatti,2200000.0
527,Aleix García,2200000.0
528,Domingos Quina,3080000.0
529,Facundo Pellistri,7700000.0


#### Serie A player market values

In [4]:
pmv_table_seriea = su.concat_table_seriea().reset_index(drop=True)
pmv_table_seriea

,Player Name,Current Market Value (USD)
0,Federico Chiesa,66000000.0
1,Nicolò Barella,66000000.0
2,Paulo Dybala,66000000.0
3,Milan Skriniar,60500000.0
4,Alessandro Bastoni,60500000.0
...,...,...
582,Joseph Marie Minala,770000.0
583,Graziano Pellè,715000.0
584,Jérôme Onguéné,4400000.0
585,Bryan Reynolds,6600000.0


#### Bundesliga player market values

In [5]:
pmv_table_bundesliga = su.concat_table_bundesliga().reset_index(drop=True)
pmv_table_bundesliga

,Player Name,Current Market Value (USD)
0,Ron-Thorben Hoffmann,660000.0
1,Carlo Boukhalfa,440000.0
2,Erling Haaland,121000000.0
3,Jadon Sancho,110000000.0
4,Joshua Kimmich,99000000.0
...,...,...
537,Joscha Wosz,440000.0
538,Hugo Novoa,220000.0
539,Robert Glatzel,2200000.0
540,Demarai Gray,13200000.0


#### Ligue 1 player market values

In [6]:
pmv_table_ligue1 = su.concat_table_ligue1().reset_index(drop=True)
pmv_table_ligue1

,Player Name,Current Market Value (USD)
0,Elye Wahi,3300000.0
1,Pape Sarr,3300000.0
2,Franco Tongya,2750000.0
3,Tom Lacoux,660000.0
4,Xavi Simons,550000.0
...,...,...
571,Aboubakar Kamara,2200000.0
572,Anthony Modeste,3080000.0
573,Pape Abou Cissé,5500000.0
574,Denis Franchi,275000.0


In [7]:
pmv_table = pd.concat([pmv_table_epl,pmv_table_laliga,pmv_table_seriea,pmv_table_bundesliga,pmv_table_ligue1]).reset_index(drop=True)

In [8]:
pmv_table.drop_duplicates(subset='Player Name', inplace=True)

In [9]:
pmv_table.reset_index(drop=True, inplace=True)

In [10]:
pmv_table

,Player Name,Current Market Value (USD)
0,Harry Kane,132000000.0
1,Mohamed Salah,121000000.0
2,Trent Alexander-Arnold,110000000.0
3,Sadio Mané,110000000.0
4,Raheem Sterling,110000000.0
...,...,...
2632,Aboubakar Kamara,2200000.0
2633,Anthony Modeste,3080000.0
2634,Pape Abou Cissé,5500000.0
2635,Denis Franchi,275000.0


In [20]:
pmv_table.to_csv('../Data/pmv_table.csv', index=False)